<a href="https://colab.research.google.com/github/YaCpotato/python/blob/master/GBDTCompareWithIris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split

from keras.utils import np_utils
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix,classification_report
import time
from sklearn import model_selection
import pandas as pd

In [78]:
iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data)
iris_df = iris_df.rename(columns={
    0: 'sepal_length',
    1: 'sepal_width',
    2: 'petal_length',
    3: 'petal_width'})
iris_df['target'] = iris.target
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [0]:
train_df, test_df = model_selection.train_test_split(iris_df, test_size=0.3) # 30%をテストに使う
train_df_y = train_df[['target']]
train_df_x = train_df.copy().drop('target', axis=1)
test_df_y = test_df[['target']]
test_df_x = test_df.copy().drop('target', axis=1)

In [0]:
import xgboost as xgb
clf = xgb.XGBClassifier()

In [65]:
%%time
clf_cv = model_selection.GridSearchCV(clf, {'max_depth': [2,4,6], 'n_estimators': [50,100,200]}, verbose=1)
clf_cv.fit(train_df_x, [i[0] for i in train_df_y.values])
print(clf_cv.best_params_, clf_cv.best_score_)

clf = xgb.XGBClassifier(**clf_cv.best_params_)
clf.fit(train_df_x, [i[0] for i in train_df_y.values])

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


{'max_depth': 2, 'n_estimators': 50} 0.9238095238095237
CPU times: user 1.04 s, sys: 75 ms, total: 1.12 s
Wall time: 1.13 s


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    1.1s finished


In [66]:
pred = clf.predict(test_df_x)
print(confusion_matrix([i[0] for i in test_df_y.values], pred))
print(classification_report([i[0] for i in test_df_y.values], pred))

[[15  0  0]
 [ 0 14  0]
 [ 0  1 15]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.93      1.00      0.97        14
           2       1.00      0.94      0.97        16

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



In [0]:
import lightgbm as lgb
params = {
    'n_estimators': 10000,  # 大きめにとっておく
}

lgb = lgb.LGBMClassifier(**params)

In [70]:
%%time
lgb.fit(train_df_x, [i[0] for i in train_df_y.values],verbose=1)

CPU times: user 1.05 s, sys: 210 ms, total: 1.26 s
Wall time: 1.28 s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=10000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [71]:
pred = clf.predict(test_df_x)
print(confusion_matrix([i[0] for i in test_df_y.values], pred))
print(classification_report([i[0] for i in test_df_y.values], pred))

[[15  0  0]
 [ 0 14  0]
 [ 0  1 15]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.93      1.00      0.97        14
           2       1.00      0.94      0.97        16

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



In [72]:
!pip install catboost

     |████████████████████████████████| 63.9MB 65kB/s 


In [0]:
from catboost import CatBoostClassifier
from catboost import Pool

train_pool = Pool(train_df_x, label=[i[0] for i in train_df_y.values])
test_pool = Pool(test_df_x, label=[i[0] for i in test_df_y.values])

In [0]:
cb = CatBoostClassifier(iterations=1000, loss_function="MultiClass",classes_count=3)

In [95]:
%%time
cb.fit(train_pool)

0:	learn: 1.0624538	total: 712us	remaining: 712ms
1:	learn: 1.0302390	total: 1.99ms	remaining: 991ms
2:	learn: 1.0041692	total: 3.19ms	remaining: 1.06s
3:	learn: 0.9717787	total: 4.13ms	remaining: 1.03s
4:	learn: 0.9434635	total: 7.04ms	remaining: 1.4s
5:	learn: 0.9146094	total: 7.76ms	remaining: 1.28s
6:	learn: 0.8886884	total: 8.59ms	remaining: 1.22s
7:	learn: 0.8669576	total: 9.34ms	remaining: 1.16s
8:	learn: 0.8418528	total: 10.1ms	remaining: 1.11s
9:	learn: 0.8187806	total: 10.9ms	remaining: 1.08s
10:	learn: 0.7973464	total: 11.6ms	remaining: 1.04s
11:	learn: 0.7798931	total: 12.3ms	remaining: 1.01s
12:	learn: 0.7620660	total: 13ms	remaining: 986ms
13:	learn: 0.7413079	total: 13.9ms	remaining: 977ms
14:	learn: 0.7207347	total: 14.5ms	remaining: 954ms
15:	learn: 0.7020698	total: 15.2ms	remaining: 936ms
16:	learn: 0.6832321	total: 15.9ms	remaining: 920ms
17:	learn: 0.6663927	total: 16.6ms	remaining: 904ms
18:	learn: 0.6505403	total: 17.4ms	remaining: 899ms
19:	learn: 0.6358767	total

In [96]:
# テストデータを予測する
Y_pred = cb.predict(test_pool, prediction_type='Class')

# 精度 (Accuracy) を計算する
from sklearn.metrics import confusion_matrix,classification_report
print (confusion_matrix(Y_test, Y_pred))
print (classification_report(Y_test, Y_pred))

[[5 5 3]
 [3 7 4]
 [5 6 7]]
              precision    recall  f1-score   support

           0       0.38      0.38      0.38        13
           1       0.39      0.50      0.44        14
           2       0.50      0.39      0.44        18

    accuracy                           0.42        45
   macro avg       0.42      0.42      0.42        45
weighted avg       0.43      0.42      0.42        45

